<a href="https://colab.research.google.com/github/Maitry-P/Python-Coding-for-Public-Policy/blob/main/Manipulating%20and%20Combining%20Data_MP02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/main/hw_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOMEWORK 2

Name: YOUR RESPONSE HERE

## Coding: Using keywords to categorize 311 requests

**Problem Statement:** When you read through the `descriptor` column in the 311 data, you will see that complaints related to graffiti are actually scattered throughout multiple `complaint_type` categories. We want to identify all complaints related to graffiti and see which community districts have the most instances of graffiti.

To help make this assignment easier, there's a smaller subset of the 311 data for you to use. It's named  `cleaned_311_data_hw2.csv` and is available at https://nyu.box.com/shared/static/i24kf9gkmc4ey506hw3oxxyaxz67k0mv.zip. This smaller dataset only contains ~65,000 records from relevant complaint type categories, and has columns renamed to be lowercase and underscored.

First, load the data:

In [ ]:
import pandas as pd
df = pd.read_csv('https://nyu.box.com/shared/static/i24kf9gkmc4ey506hw3oxxyaxz67k0mv.zip')

Preview the dataframe:

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11434.0,NaN,NaN,NaN,NaN,BAISLEY BOULEVARD,MERRICK BOULEVARD,INTERSECTION,JAMAICA,NaN,NaN,Closed,08/04/2018 03:20:07 AM,The Service Request submitted did not have suf...,08/23/2018 11:15:00 AM,12 QUEENS,NaN,QUEENS,1048111.0,188479.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)"
2,2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,11206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,Closed,08/04/2018 04:19:15 AM,The Department of Parks and Recreation has com...,08/02/2018 08:19:55 AM,01 BROOKLYN,NaN,BROOKLYN,NaN,NaN,MOBILE,Ten Eyck Plaza,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAR ROCKAWAY,NaN,NaN,Closed,08/04/2018 07:32:33 AM,The Department of Parks and Recreation has com...,08/16/2018 10:28:38 AM,14 QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Westbourne Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLUSHING,NaN,NaN,Closed,08/04/2018 07:33:12 AM,The condition was inspected and it was determi...,09/04/2018 05:38:02 PM,07 QUEENS,NaN,QUEENS,NaN,NaN,MOBILE,Maple Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 1

Create a function that checks each row in the 311 dataframe to see if the word "graffiti" is present in either the `complaint_type` value or `descriptor` value. Both columns may contain the word, so you should check both. If the word "graffiti" is found, the function should return the boolean value True. If "graffiti" is not found, the function should return the boolean value False.

#### Hints

- The same way that we checked if the string "BRONX" existed in the `community_board` value in the `cb_counts` table during the lecture, you can check if "graffiti" exists in the `descriptor` or `complaint_type` values.
- The `descriptor` column contains some null/NaN values. You'll have trouble running a function on these. Before your function checks if "graffiti" is present in `descriptor`, the function should make sure that a `descriptor` value exists using `pd.notnull(row.descriptor)`
- Capitalization may be inconsistent. It could help to use `.lower()` to convert strings to lowercase.

In [ ]:
def flag_graffiti(row):
  if 'graffiti' in row.complaint_type.lower():
    return True
  elif pd.notnull(row.descriptor):
    if 'graffiti' in row.descriptor.lower():
      return True
    else:
      return False
  else:
    return False

### Step 2

Apply the function created in Step 1 to the 311 dataframe and create a new column called `graffiti_flag` that captures the output from the function.

Tip: There are two checks you can use to confirm that the function worked as expected.

- Group by `graffiti_flag` to make sure there are records tagged as `True`.
- Using only the records where `graffiti_flag` is `True` (`sample[sample.graffiti_flag]`), group by `complaint_type` to make sure that more than one `complaint_type` is included.

In [ ]:
df['graffiti_flag'] = df.apply(flag_graffiti, axis=1)
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,graffiti_flag
0,0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11434.0,NaN,NaN,NaN,NaN,BAISLEY BOULEVARD,MERRICK BOULEVARD,INTERSECTION,JAMAICA,NaN,NaN,Closed,08/04/2018 03:20:07 AM,The Service Request submitted did not have suf...,08/23/2018 11:15:00 AM,12 QUEENS,NaN,QUEENS,1048111.0,188479.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)",False
2,2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,11206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,Closed,08/04/2018 04:19:15 AM,The Department of Parks and Recreation has com...,08/02/2018 08:19:55 AM,01 BROOKLYN,NaN,BROOKLYN,NaN,NaN,MOBILE,Ten Eyck Plaza,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAR ROCKAWAY,NaN,NaN,Closed,08/04/2018 07:32:33 AM,The Department of Parks and Recreation has com...,08/16/2018 10:28:38 AM,14 QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Westbourne Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLUSHING,NaN,NaN,Closed,08/04/2018 07:33:12 AM,The condition was inspected and it was determi...,09/04/2018 05:38:02 PM,07 QUEENS,NaN,QUEENS,NaN,NaN,MOBILE,Maple Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
df.groupby('graffiti_flag').size()


graffiti_flag
False    38206
True     26371
dtype: int64

In [ ]:
df.groupby('complaint_type').size()

complaint_type
Bridge Condition                392
Broken Parking Meter          14210
Bus Stop Shelter Complaint      990
Graffiti                      24986
Highway Condition              3543
Maintenance or Facility       17665
Public Payphone Complaint       378
Public Toilet                    41
School Maintenance             2372
dtype: int64

### Step 3

Create another dataframe `df_graffiti` that only contains records where `graffiti_flag` is `True`. `df_graffiti = df[df.graffiti_flag]`

In [ ]:
df_graffiti = df[df['graffiti_flag']]
len(df_graffiti)

26371

### Step 4

Group your dataframe `df_graffiti` to get the count of requests per `community_board`. Use `.nlargest()` to identify which Community District has the highest count.

In [ ]:
df_graffiti.groupby('community_board').size().nlargest(15)

community_board
01 BROOKLYN             1797
03 MANHATTAN            1593
04 BROOKLYN             1429
03 BROOKLYN             1252
05 BROOKLYN              962
07 BROOKLYN              779
Unspecified BRONX        615
02 MANHATTAN             598
Unspecified BROOKLYN     591
01 QUEENS                572
0 Unspecified            540
12 QUEENS                539
12 BROOKLYN              532
14 BROOKLYN              532
08 BROOKLYN              505
dtype: int64

Brooklyn has the highest count of graffiti complaints.

### Bonus

Create a `graffiti_flag2` column using only built-in pandas operations, i.e. without using a custom function.

In [ ]:
# Option:1
def recode_borocd_counts(row):
  if 'MANHATTAN' in row["community_board"]:
    return '1' + row["community_board"][0:2]
    # [0:2] provides the first 2 characters, i.e. characters at indexes 0 and 1.
    # you could also use [:2] without the zero.
  elif 'BRONX' in row["community_board"]:
    return '2' + row["community_board"][0:2]
  elif 'BROOKLYN' in row["community_board"]:
    return '3' + row["community_board"][0:2]
  elif 'QUEENS' in row["community_board"]:
    return '4' + row["community_board"][0:2]
  elif 'STATEN ISLAND' in row["community_board"]:
    return '5' + row["community_board"][0:2]
  else:
    return 'Invalid BoroCD'
type(recode_borocd_counts)

function

In [ ]:
df['graffiti_flag2'] = df.apply(recode_borocd_counts, axis=1)
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,graffiti_flag,graffiti_flag2
0,0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Invalid BoroCD
1,1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11434.0,NaN,NaN,NaN,NaN,BAISLEY BOULEVARD,MERRICK BOULEVARD,INTERSECTION,JAMAICA,NaN,NaN,Closed,08/04/2018 03:20:07 AM,The Service Request submitted did not have suf...,08/23/2018 11:15:00 AM,12 QUEENS,NaN,QUEENS,1048111.0,188479.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)",False,412
2,2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,11206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,Closed,08/04/2018 04:19:15 AM,The Department of Parks and Recreation has com...,08/02/2018 08:19:55 AM,01 BROOKLYN,NaN,BROOKLYN,NaN,NaN,MOBILE,Ten Eyck Plaza,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,301
3,3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAR ROCKAWAY,NaN,NaN,Closed,08/04/2018 07:32:33 AM,The Department of Parks and Recreation has com...,08/16/2018 10:28:38 AM,14 QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Westbourne Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,414
4,4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLUSHING,NaN,NaN,Closed,08/04/2018 07:33:12 AM,The condition was inspected and it was determi...,09/04/2018 05:38:02 PM,07 QUEENS,NaN,QUEENS,NaN,NaN,MOBILE,Maple Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64572,64572,2857911,2858100,43618118,08/23/2019 10:44:09 PM,NaN,DPR,Department of Parks and Recreation,Maintenance or Facility,Rodent Sighting,Park,10034.0,570 WEST 214 STREET,WEST 214 STREET,SEAMAN AVENUE,INDIAN ROAD,SEAMAN AVENUE,INDIAN ROAD,NaN,NEW YORK,WEST 214 STREET,NaN,In Progress,NaN,NaN,NaN,12 MANHATTAN,1.022550e+09,MANHATTAN,1006555.0,256695.0,ONLINE,Isham Park,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.871215,-73.919351,"(40.87121476286591, -73.91935095259326)",False,112
64573,64573,2857942,2858131,43623672,08/23/2019 10:50:40 PM,NaN,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10011.0,WEST 20 STREET,WEST 20 STREET,6 AVENUE,7 AVENUE,6 AVENUE,7 AVENUE,NaN,NaN,NaN,NaN,In Progress,NaN,NaN,NaN,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Invalid BoroCD
64574,64574,2857951,

In [ ]:
#Option 2
def recode_borocd_counts(row):
    board = row["community_board"]

    if board[0] != 'U':
        num = board[0:2]
        
        if 'MANHATTAN' in board:
            return '1' + num
        elif 'BRONX' in board:
            return '2' + num
        elif 'BROOKLYN' in board:
            return '3' + num
        elif 'QUEENS' in board:
            return '4' + num
        elif 'STATEN ISLAND' in board:
            return '5' + num
    else:
        return 'Invalid BoroCD'
df['graffiti_flag2'] = df.apply(recode_borocd_counts, axis=1)
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,graffiti_flag,graffiti_flag2
0,0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None
1,1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11434.0,NaN,NaN,NaN,NaN,BAISLEY BOULEVARD,MERRICK BOULEVARD,INTERSECTION,JAMAICA,NaN,NaN,Closed,08/04/2018 03:20:07 AM,The Service Request submitted did not have suf...,08/23/2018 11:15:00 AM,12 QUEENS,NaN,QUEENS,1048111.0,188479.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)",False,412
2,2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,11206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,Closed,08/04/2018 04:19:15 AM,The Department of Parks and Recreation has com...,08/02/2018 08:19:55 AM,01 BROOKLYN,NaN,BROOKLYN,NaN,NaN,MOBILE,Ten Eyck Plaza,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,301
3,3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAR ROCKAWAY,NaN,NaN,Closed,08/04/2018 07:32:33 AM,The Department of Parks and Recreation has com...,08/16/2018 10:28:38 AM,14 QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Westbourne Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,414
4,4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLUSHING,NaN,NaN,Closed,08/04/2018 07:33:12 AM,The condition was inspected and it was determi...,09/04/2018 05:38:02 PM,07 QUEENS,NaN,QUEENS,NaN,NaN,MOBILE,Maple Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64572,64572,2857911,2858100,43618118,08/23/2019 10:44:09 PM,NaN,DPR,Department of Parks and Recreation,Maintenance or Facility,Rodent Sighting,Park,10034.0,570 WEST 214 STREET,WEST 214 STREET,SEAMAN AVENUE,INDIAN ROAD,SEAMAN AVENUE,INDIAN ROAD,NaN,NEW YORK,WEST 214 STREET,NaN,In Progress,NaN,NaN,NaN,12 MANHATTAN,1.022550e+09,MANHATTAN,1006555.0,256695.0,ONLINE,Isham Park,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.871215,-73.919351,"(40.87121476286591, -73.91935095259326)",False,112
64573,64573,2857942,2858131,43623672,08/23/2019 10:50:40 PM,NaN,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10011.0,WEST 20 STREET,WEST 20 STREET,6 AVENUE,7 AVENUE,6 AVENUE,7 AVENUE,NaN,NaN,NaN,NaN,In Progress,NaN,NaN,NaN,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None
64574,64574,2857951,2858140,43619750,08/

Now [turn in the assignment](https://github.com/afeld/python-public-policy#turning-in-assignments).

## Tutorials

- ["You’re Not Mapping Rats, You're Mapping Gentrification"](https://theconcourse.deadspin.com/you-re-not-mapping-rats-you-re-mapping-gentrification-1835005060)—article about bias in 311 data
- [Intro to Plotly Express](https://plotly.com/python/plotly-express/). You don't have to work through every one of these examples; just review to get familiar with what types of charts are possible.